In [ ]:
from openai import AzureOpenAI
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.text_splitter import  RecursiveCharacterTextSplitter,  CharacterTextSplitter

from langchain_community.document_loaders import JSONLoader, DirectoryLoader
from langchain_community.document_loaders import UnstructuredExcelLoader,  UnstructuredPowerPointLoader, Docx2txtLoader, PyPDFLoader,  UnstructuredWordDocumentLoader, TextLoader, UnstructuredFileLoader

from langchain.chains import RetrievalQA

from langchain.retrievers.merger_retriever import MergerRetriever
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain_experimental.text_splitter import SemanticChunker
import pandas as pd
import json 

In [ ]:
open_ai_type = "azure"
open_ai_api_version = "2023-05-15"
open_ai_api_key = "open_ai_api_key"
open_ai_api_key = open_ai_api_key 
open_ai_endpoint = "open_ai_endpoint" # base
embeddings_model = "janus-text-embedding-ada-002"
gpt_deployment_4 = "janus-gpt-4"
gpt_deployment_35 = "janus-gpt-35-turbo"
gpt_deployment_35_16="janus-gpt-35-turbo-16k"
embeddings_model = "janus-text-embedding-ada-002"
vector_store_endpoint = "vector_store_endpoint"
vector_store_key = "vector_store_key"
vector_index_name = "vector_index_name"

In [ ]:
#from azure.search.documents.indexes import SearchIndexClient
#from azure.search.documents.indexes.models import (
#     ComplexField,
#     SearchableField,
#     SimpleField,
#     SearchIndex,
#     SearchField,
    
# )
from azure.core.credentials import AzureKeyCredential
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  

# # Replace with your search service name and admin key
# service_name = "your-service-name"
# admin_key = "your-admin-key"

# # Replace with your service endpoint
# endpoint = f"https://{service_name}.search.windows.net"
# index_name = "your-index-name"

# Create a SearchIndexClient
client = SearchIndexClient(vector_store_endpoint, AzureKeyCredential(vector_store_key))

##  To create the index

In [ ]:


fields=[
        SimpleField(name="id",type=SearchFieldDataType.String , retrievable=True, key=True, sortable= True, filterable=True, facetable=True),
        SearchField(name="incoming_email_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single) ,searchable=True,vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile" ),
        SearchableField(name="incoming_email",type=SearchFieldDataType.String, searchable=True, retrievable=True, filterable=True, facetable=True),
        SearchableField(name="outgoing_email", type=SearchFieldDataType.String, searchable=True, retrievable=True, filterable=True, facetable=True)
        

         
        #ComplexField(name="incoming_email_embedding", type=SearchFieldDataType.COLLECTION(SearchFieldDataType.SINGLE)),
        #ComplexField(name="outgoing_email_embedding", type=SearchFieldDataType.COLLECTION(SearchFieldDataType.SINGLE))
        # SimpleField(name="id", type="Edm.String", retrievable=True, key=True, sortable= True, filterable=True, facetable=True),
        # SearchableField(name="incoming_email_vector", type="Collection(Edm.Double)", searchable=True, retrievable=True,filterable=True, facetable=True),
        # SearchableField(name="incoming_email", type="Edm.String", searchable=True, retrievable=True, filterable=True, facetable=True),
        # SearchableField(name="outgoing_email", type="Edm.String", searchable=True, retrievable=True, filterable=True, facetable=True),
        # SearchField(name="embedding_vector", type= "SearchFieldDataType Collection(Edm.Double)",searchable=True,vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile" )
    ]



In [ ]:
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        )
    ]
)

In [ ]:
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        content_fields=[SemanticField(field_name="incoming_email")],
        keywords_fields=[SemanticField(field_name="outgoing_email")]
    )
)

In [ ]:
# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=vector_index_name, fields=fields,
                    vector_search=vector_search, 
                    semantic_search=semantic_search)
result = client.create_or_update_index(index)
print(f' {result.name} created')

## Code to ingest the JSON into index created above

In [ ]:
from azure.search.documents import SearchClient
# Open and read the 'data.json' file, which contains the documents to be uploaded and queried
with open('New_mail_for_index.json', 'r') as file:
    documents = json.load(file)

# Create a Search Client instance for uploading and querying data
search_client = SearchClient(vector_store_endpoint, vector_index_name, credential=AzureKeyCredential(vector_store_key))

# Upload the documents to the specified search index using the Search Client
#result = search_client.upload_documents(documents)
def split_documents(documents, batch_size):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]

batch_size = 100  # Adjust the batch size as needed

try:
    for batch in split_documents(documents, batch_size):
        result = search_client.upload_documents(batch)
        print(f"Uploaded {len(batch)} documents")
except Exception as e:
    print(f"An error occurred: {e}")

# Print the number of documents successfully uploaded
#print(f"Uploaded {len(documents)} documents")

## The Code to create JSON  

In [ ]:
index = client.get_index(vector_index_name)
# Print the fields in the index
for field in index.fields:
    print(f"Field name: {field.name}, Field type: {field.type}")

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=embeddings_model,
    openai_api_version=open_ai_api_version,
    azure_endpoint = open_ai_endpoint,
    openai_api_key = open_ai_api_key
)

In [ ]:
# Define a function to get text embeddings using OpenAI's text-embedding model.
def get_embedding(text,embeddings):
    # Replace newline characters with spaces in the input text.
    text = text.replace("\n", " ")
    
    # Call OpenAI's text-embedding API to obtain embeddings for the input text.
    #embeddings = langchain_openai.Embedding.create(input=[text], model=model)
    Embedding = embeddings.embed_query(text)
    # Extract the embedding vector from the API response and return it.
    embedding_vector = Embedding
    return embedding_vector



In [ ]:
df= pd.read_excel(r"Your_path.xlsx")

In [ ]:
df.columns

In [ ]:
df1=pd.DataFrame()

In [ ]:
df1["Incoming_emails"]=df["Cleansed_Incoming_Email_Masked"]
df1["Outgoing_emails"]=df["Cleansed_Outgoing_Email_Masked"]

In [ ]:
df1.head()

In [ ]:
df1.to_excel("New_mail_for_index.xlsx")

In [ ]:
data= pd.read_excel("New_mail_for_index.xlsx")

In [ ]:
data["Incoming_emails_embedding"] = data.Incoming_emails.apply(lambda x: get_embedding(x, embeddings ))

In [ ]:
# DataFrame with assigned columns
assigned_df = data.assign( 
    id = data["Unnamed: 0"],
    incoming_email_vector =data["Incoming_emails_embedding"],
    incoming_email =data["Incoming_emails"],
    outgoing_email =data["Outgoing_emails"]
)

# DataFrame with index columns
filtered_df = assigned_df.drop(
    ["Unnamed: 0", "Incoming_emails_embedding", "Incoming_emails", "Outgoing_emails"],
    inplace=False,
    axis=1
)

In [ ]:
filtered_df.head()

In [ ]:
filtered_df["id"]= filtered_df["id"].astype(str)

In [ ]:
filtered_df.to_json("New_mail_for_index.json", orient='records')